# Computer Project for TMA4215

$$
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Rot}{rot}
\DeclareMathOperator{\ord}{ord}
\DeclareMathOperator{\Kern}{ker}
\DeclareMathOperator{\Image}{im}
\DeclareMathOperator{\spann}{span}
\DeclareMathOperator{\rank}{rank}
\DeclareMathOperator{\dist}{dist}
\DeclareMathOperator{\diam}{diam}
\DeclareMathOperator{\sig}{sig}
\DeclareMathOperator{\Id}{Id}
\DeclareMathOperator{\CQR}{CQR}
\DeclareMathOperator{\QR}{QR}
\DeclareMathOperator{\TR}{TR}
\DeclareMathOperator{\CTR}{CTR}
\DeclareMathOperator{\SR}{SR}
\DeclareMathOperator{\CSR}{CSR}
\DeclareMathOperator{\NCR}{NCR}
\DeclareMathOperator{\MR}{MR}
\newcommand{\RR}{\mathbb{R}}
\newcommand{\NN}{\mathbb{N}}
\newcommand{\VV}{\mathbb{V}}
\newcommand{\PP}{\mathbb{P}}
\newcommand{\dGamma}{\,\mathrm{d} \Gamma}
\newcommand{\dGammah}{\,\mathrm{d} \Gamma_h}
\newcommand{\dx}{\,\mathrm{d}x}
\newcommand{\dy}{\,\mathrm{d}y}
\newcommand{\ds}{\,\mathrm{d}s}
\newcommand{\dt}{\,\mathrm{d}t}
\newcommand{\dS}{\,\mathrm{d}S}
\newcommand{\dV}{\,\mathrm{d}V}
\newcommand{\dX}{\,\mathrm{d}X}
\newcommand{\dY}{\,\mathrm{d}Y}
\newcommand{\dE}{\,\mathrm{d}E}
\newcommand{\dK}{\,\mathrm{d}K}
\newcommand{\dM}{\,\mathrm{d}M}
\newcommand{\cd}{\mathrm{cd}}
\newcommand{\onehalf}{\frac{1}{2}}
\newcommand{\bfP}{\boldsymbol P}
\newcommand{\bfx}{\boldsymbol x}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bfa}{\boldsymbol a}
\newcommand{\bfu}{\boldsymbol u}
\newcommand{\bfv}{\boldsymbol v}
\newcommand{\bfe}{\boldsymbol e}
\newcommand{\bfb}{\boldsymbol b}
\newcommand{\bfc}{\boldsymbol c}
\newcommand{\bfq}{\boldsymbol q}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bff}{\boldsymbol f}
\newcommand{\bfp}{\boldsymbol p}
\newcommand{\bft}{\boldsymbol t}
\newcommand{\bfj}{\boldsymbol j}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfV}{\boldsymbol V}
\newcommand{\bfE}{\boldsymbol E}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfzero}{\boldsymbol 0}
$$

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/tma4215.css", "r").read()
    return HTML(styles)

# Comment out next line and execute this cell to restore the default notebook style 
#css_styling()

The overall goal of this project is to develop a numerical solver for the following nonlinear diffusion-reaction problems which can stated as follows.

Find a function u(x,y,t) satisfying the initial value problem
\begin{alignat}{3}
\partial_t u - \kappa \Delta u + c(u) u &= f
&& \quad \text{in } \Omega  \times (t_0, T)
\subset \RR^2 \times \RR,
\tag{1a}
\\
u &= u_D && \quad  \text{on } \partial \Omega  \times (t_0, T)
\subset \RR^2 \times \RR
\tag{1b}
\\
u(\cdot,t_0) &= u_0
&&\quad \text{in } \Omega,
\tag{1c}
\end{alignat}

for some domain $\Omega \subset \RR^2$ and some time interval $[t_0, T] \subset \RR$.

As usual, $\Delta u$ denotes the usual Laplace operator
$$
\Delta u(x,y,t) = 
\partial^2_{x} u(x,y,t) + \partial^2_{y} u(x,y,t)
= 
\dfrac{\partial^2}{\partial x^2} u(x,y,t) 
+\dfrac{\partial^2}{\partial y^2} u(x,y,t) 
$$
and $\partial_t u(x,y,t = \dfrac{\partial}{\partial t} u(x,y,t)$.

Observe that the entire problem consists of the partial differential equation (1a)
which is supplemented with some boundary condition at any relevant time, (1b)
and some initial data given in (1c).

Such and  related problems describe for instance (as the name suggest)  diffusion and reaction in chemical systems, but occurr
also in biology (pattern development on fur, chemotactial motion of cells) and in many other fields. The term $-\kappa \Delta u$ typically models the
diffusion of some quantity, e.g. energy, chemical substance over time,
while the term $c(u) u$ typically models the change of the quantity due to some reaction. Without the term $c(u)$ this would be the standard heat equation.

The development of a numerical method for (1a)-(1c) will be divided into 3 steps:

* First, we neglect time and the non-linear term $c(u)u$, and study how to discretize a stationary problem known as the __Poisson problem__
\begin{alignat*}{3}
-\Delta u &= f &&\quad \text{in } \Omega, 
\\
u &= u_D  && \quad\text{on } \partial \Omega. 
\end{alignat*} 
At the end we will be left with a task to solve the linear system $AU = F$.
* Next, we develop a solver for the __classical heat equation__
\begin{alignat*}{3}
\partial_t u - \kappa \Delta u &= f
&& \quad \text{in } \Omega  \times (t_0, T)
\subset \RR^2 \times \RR,
\\
u &= u_D && \quad  \text{on } \partial \Omega  \times (t_0, T)
\subset \RR^2 \times \RR
\\
u(\cdot,t_0) &= u_0
&&\quad \text{in } \Omega,
\end{alignat*}
ignoring the non-linear term $c(u)u$. 
By using the techniques from step 1, we will end up with the task of
solving an ODE system of the form
$$
\dot{U} + A U = F
$$
and then apply certain time-discretization techniques. 
* Finally, we will apply similar techniques to reduce our original problem
to a non-linear ODE
$$
\dot{U} + A U + C(U)U = F
$$
and will combine time-stepping techniques with the solution techniques for non-linear
equations we learned about during this course (Newton's method and fix-point iteration schemes).

## Part 1: The Poisson Equation


### Chapter 1: The two-point boundary value problem

Let $[a,b] \subset \RR$ be a finite interval, together
with a function $f: [a, b] \to \RR$. Then the _two-point boundary value problem_ is to find a $u: [a, b] \to \RR$
such that

\begin{gather}
- u''(x) = f(x) \quad \forall x \in (a,b),
\tag{1a}
\\
u(a) = u_a, \quad  u(b) = u_b
\tag{1b}
\end{gather}
where $u_a$ and $u_b$ are given values.

__For the rest of the the project we will 
assume that $a = 0, b = 1$ and $u_a = u_b = 0$.__

Note that is a __second order__ ordinary differential equation and thus requires two addition values to determine a solution completely, one on each boundary point.

### Finite difference method: A way to solve the two-point  boundary problem numerically
Instead of trying to compute $u(x)$ exactly,
we will try to compute a numerical approximation
$u_{\Delta}$ of $u(x)$. As many times before, we start by 
defining $n+1$ equally spaced points $\{x_i\}_{i=0}^n$ 
with a grid size $h = \tfrac{b-a}{n}$ such that
$$
x_i := a + i h \quad
\text{for } i = 0, 1, \ldots, n.
$$

Then the second order derivative $u''$ can be approximated by the
__central difference operator__ defined by

$$
\begin{align}
\partial^+ \partial^- u(x)
:=  \dfrac{u(x+h) - 2 u(x) + u(x-h)}
{h^2}
\approx u''(x) = -f(x)
\end{align}
$$

Recall that for  $u \in C^k([0,1])$, the 
Taylor expansion of $u$ around $x$ is given by
$$
u(x+h) = u(x) + h u'(x) + \dfrac{h^2}{2!} u''(x) 
%+\dfrac{h^3}{3!} u^{(3)}(x)
+ \ldots 
+ \dfrac{h^{k-1}}{(k-1)!} u^{(k-1)}(x)
+ \dfrac{h^{k}}{k!} u^{(k)}(\xi)
$$
for some $\xi \in (x,x+h)$. 
Since $u \in C^k([0,1])$, 
the remainder term  $\dfrac{h^k}{k!} u^{(k)}(\xi)$ is uniformly bounded with respect to $\xi$; that is,
$$
\max_{\xi \in [a,b]}|u^{(k)}(\xi)| \leqslant C.
$$

Thus we can simply write
$$
u(x+h) = u(x) + h u'(x) + \dfrac{h^2}{2!} u''(x) 
%+\dfrac{h^3}{3!} u^{(3)}(x)
+ \ldots 
+ \dfrac{h^{k-1}}{(k-1)!} u^{(k-1)}(x)
+ \mathcal{O}(h^{k})
$$

#### Task 1
Use Taylor expansion to show that
for 
$u \in C^4([0,1])$

$$
\max_{x \in [a,b]} | \partial^+ \partial^-u(x) - u''(x) |
=
\mathcal{O}(h^2).
$$

<font color="blue"> 
<b>Solution:</b><br>
For $u \in C^4([0, 1])$ we get that

$$
\begin{equation}
    \begin{split}
        u(x + h) - 2u(x) + u(x - h) = \quad \quad & \\
        u(x) + hu'(x) + \frac{h^2}{2!}u''(x) + \frac{h^3}{3!}u'''(x) & \\
        + u(x) - hu'(x) + \frac{h^2}{2!}u''(x) - \frac{h^3}{3!}u'''(x) & - 2u(x) + \mathcal{O}(h^4) & \\
        = 2u(x) + h^2u''(x) - 2u(x) + \mathcal{O}(h^4) \quad \\
    \end{split}
\end{equation}
$$

Inserting this into the equation above, we get

$$
\begin{equation}
    \begin{split}
        \max_{x \in [a,b]} |\partial^+ \partial^- u(x) - u''(x)| = \max_{x \in [a,b]} \left|\frac{h^2 u''(x) + \mathcal{O}(h^4)}{h^2} - u''(x)\right| \\
        \quad = \max_{x \in [a,b]} \left|u''(x) + \frac{\mathcal{O}(h^4)}{h^2} - u''(x)\right|  = \mathcal{O}(h^2) \\
    \end{split}
\end{equation}
$$

</font>

Now the idea is to numerically solve for the two-point boundary 
value problem by associating an unknown variable $U_i := u_{\Delta}(x_i)$ to every 
grid point $x_i$ for $i=0,\ldots N$. The goal
is to find an $U_i \approx u(x_i)$. Keeping in mind that $-u'' = f$,
we demand that at the __internal__ grid points $\{x_i\}_{i=1}^{N-1}$
the unknows satisfy

\begin{align}
-\partial^+\partial^- U_i
=
- \dfrac{U_{i+1} - 2 U_i + U_{i-1}}
{h^2}
= f(x_i) \quad \text{for } i = 1,\ldots N-1.
\end{align}

Note that at $x_0$ and $x_N$, $\partial^+\partial^- U_i$ is not well-defined
since we don't have a point left (respective right) of $x_0$ (respective $x_N$). This leads to the $N-1 \times N+1$ linear system of the form

\begin{align}
\dfrac{1}{h^2}
\underbrace{
\begin{bmatrix}
-1 & 2  & -1 & & &
\\
  & -1 & 2  & -1 & & 
\\
  &    & -1 & 2 & -1 & & 
\\
 &     &    & \ddots &\ddots & \ddots &
\\
&      &    &        &  -1 & 2 & -1
\end{bmatrix}
}_{\widetilde{A}}
\underbrace{
\begin{bmatrix}
U_0
\\
U_1
\\
U_2
\\
U_3
\\
\vdots
\\
U_{N-1}
\\
U_N
\end{bmatrix}
}_{{U}}
=
\underbrace{
\begin{bmatrix}
f(x_1)
\\
f(x_2)
\\
f(x_3)
\\
\vdots
\\
f(x_{N-1})
\end{bmatrix}
}_{\widetilde{F}}
\end{align}

We immediately ask whether this system
is _always solvable_ and has a _unique solution_.
Unfortunately the matrix $\widetilde{A}$ above has a nontrivial kernel,
for instance $U_0 = U_1 = U_2 = \ldots U_{N-1} = U_N = c$ for any constant $c$. That's where the boundary conditions come into play!

To close the system, we incorporate the boundary condition by setting
$$
U_0 = u_a, \quad U_{N} = u_b.
$$
These trivial equations can be added to the system above,
leading to final problem:

Find $U = [U_0, \ldots, U_N] \in \RR^{N+1}$ such that

\begin{align}
\dfrac{1}{h^2}
\underbrace{
\begin{bmatrix}
h^2
\\
1 & 2  & -1 & & &
\\
  & -1 & 2  & -1 & & 
\\
  &    & -1 & 2 & -1 & & 
\\
 &     &    & \ddots &\ddots & \ddots &
\\
&      &    &        &  -1 & 2 & -1
\\
&      &    &        &  &  & h^2
\end{bmatrix}
}_{{A}}
\underbrace{
\begin{bmatrix}
U_0
\\
U_1
\\
U_2
\\
U_3
\\
\vdots
\\
U_{N-1}
\\
U_{N}
\end{bmatrix}
}_{{U}}
=
\underbrace{
\begin{bmatrix}
u_a
\\
f(x_1)
\\
f(x_2)
\\
f(x_3)
\\
\vdots
\\
f(x_{N-1})
\\
u_b
\end{bmatrix}
}_{{F}}
\end{align}

#### Task 2
Implement a Python function ```fdm_poisson1d_matrix(N)``` which computes the finite difference
matrix for $N+1$ equally spaced points $\{x_i\}_{i=0}^{N}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def fdm_poisson1d_matrix(N):
    #Gridsize
    h = 1.0/N
    
    #h squared
    hh = h*h
    
    #Create A
    A  = np.diag([2 for i in range(N+1)]) + np.diag([-1 for i in range(N)],1) +  np.diag([-1 for i in range(N)],-1)
    A[0,1], A[N,-2] = 0, 0
    A = 1/hh * A
    A[0,0], A[N,N] = 1, 1
    return A

Also, define a function ```def apply_bcs(F, bcs)``` which incorporates the proper boundary values as described above, taking a vector $F$ and a list ```bcs = [u_a, u_b]```, modifying $F$
accordingly.

In [ ]:
def apply_bcs(F, bcs):
    F=np.array(F) #in case F not array 
    return np.hstack([bcs[0],F,bcs[1]])

#### Task 3
Now, you should test your implementation by solving the two-point boundary problem (1a)--(1b) numerically.

Apply the method of __manufactured solutions__: Take a known function $u(x)$, here $\sin(2\pi x)$ and 
compute corresponding data $f$ and $u_a$, $u_b$ such that $u$ is the solution to 
the two-point two-point boundary problem (1a)--(1b).

With $u(x) = \sin(2\pi x)$, we have that $f = - u'' = 4\pi^2 \sin(2\pi x)$,
$u_a = u(0) = 0$ and $u_b = u(1) = 0$.

The error can then be computed by
$$
\mathrm{err(h)} = \mathrm{err(N)} = \max_{i \in \{0,\ldots N\}}|U_i - u(x_i)|
$$
as a function of $h = 1/N$.

For $N_1 = 5, N_2= 10, N_3 = 20, N_4 = 40, N_5 =80$
* compute and plot the numerical solution and the analytical solution
* record the resulting $\mathrm{err(N)}$

At the end , compute the __experimental order of convergence__ 
for each refinement level $N_2, \ldots, N_5$ defined by
$$
EOC(N_i) = \dfrac{\log(\mathrm{err}(N_i)) - \log(\mathrm{err}(N_{i-1}))}
{\log(N_{i-1}) - \log(N_i)}
$$

What order of convergence does this give?

In [ ]:
%matplotlib notebook

# Import linear algebra module
import scipy.linalg as la

# Analytical reference solution
u = lambda x: np.sin(2*np.pi*x)
# Right-hand side
f = lambda x: np.sin(2*np.pi*x)*(2*np.pi)**2
# Boundary conditions as list
bcs = [0,0]

# Number of subintervals = number of mesh points - 1
# Collect grid sizes and errors
hs = []
errs = []
x1 = np.array([i/(80) for i in range(0,80+1)]) 
real_sol1=u(x1)

N_list = [5, 10, 20, 40, 80]

for N in N_list:

    # Define grid points
    x = np.array([i/(N) for i in range(0,N+1)]) 

    # Compute A and F
    A = fdm_poisson1d_matrix(N)
    F = f(x[1:N])

    # Incorporate bcs 
    F = apply_bcs(F,bcs)
    # Solve AU = F
    U = la.solve(A, F)
    # Compute real solution and error at grid points and maximum error
    real_sol=u(x)
    
    hs.append(1/N)
    err=abs(U-real_sol)
    errs.append(max(err))
    
    # Plot discrete and analytical solution on chosen discretization grid
    plt.figure()
    plt.plot(x,U,label="Discrete solution")
    plt.plot(x1,real_sol1,linestyle='dashed',label="Analytic solution")
    plt.title('N= {}'.format(N))
    plt.legend()
    plt.show()

<font color = blue> The plots show that the numerical solution gives results similar to the analytic solution, and the result i better for higher n. This is as expected as a higher n gives smaller steps h which in turn gives smaller higher bound for the error. </font>

In [ ]:
# Compute eoc
eoc=[]
for i in range(1,len(errs)):
    eoc.append((np.log(errs[i])-np.log(errs[i-1]))/(np.log(N_list[i-1])-np.log(N_list[i])))
print("Experimental order of convergence: ", eoc)

<font color = blue>
We can clearly see that the experimental orders of convergence are close to 2, which is as expected from the calculations in task 1.
</font>

#### Outlook

Before we turn to the more interesting 2D problem, we have a quick look at a slightly modified version of the two-point problem (1a)-(1b).
Later when you are asked to solve some time-dependent problem,
you will have to solve at each time-step with time-step size $\tau$
a problem similar to the form

\begin{gather}
u - \tau u''(x) = f(x) \quad \forall x \in (a,b),
\tag{3a}
\\
u(a) = u_a, \quad  u(b) = u_b
\tag{2b}
\end{gather}
where $u_a$ and $u_b$ are given values.
This problem can also be solved numerically using the finite difference method resulting in the problem in the system of equations

\begin{align}
U_i 
- \tau  \dfrac{U_{i+1} - 2 U_i + U_{i-1}}
{h^2}
= f(x_i) \quad \text{for } i = 1,\ldots N-1.
\end{align}
The boundary condition can be incorporated as before,
leading to the problem

Find $U = [U_0, \ldots, U_N] \in \RR^{N+1}$ such that
$$
(\Id + \tau A) U = F 
$$
or in more verbose form,
\begin{align*}
\left(
\begin{bmatrix}
1
\\
 & 1  &  & & &
\\
  &  & 1  & & & 
\\
  &    &  &  1&  & & 
\\
 &     &    &  &\ddots & &
\\
&      &    &        &   & 1 & 
\\
&      &    &        &  &  & 1
\end{bmatrix}
+
\dfrac{\tau}{h^2}
\begin{bmatrix}
0
\\
1 & 2  & -1 & & &
\\
  & -1 & 2  & -1 & & 
\\
  &    & -1 & 2 & -1 & & 
\\
 &     &    & \ddots &\ddots & \ddots &
\\
&      &    &        &  -1 & 2 & -1
\\
&      &    &        &  &  & 0
\end{bmatrix}
\right)
\begin{bmatrix}
U_0
\\
U_1
\\
U_2
\\
U_3
\\
\vdots
\\
U_{N-1}
\\
U_{N}
\end{bmatrix}
=
\begin{bmatrix}
u_0
\\
f(x_1)
\\
f(x_2)
\\
f(x_3)
\\
\vdots
\\
f(x_{N-1})
\\
u_b
\end{bmatrix}
\end{align*}

Here $\Id \in \RR^{N+1,N+1}$ is the identity matrix, $F$ is defined as before __including__ the boundary conditions,
but $A \in \RR^{N+1,N+1}$ corresponds to the finite difference matrix
__without the modification__ to incorporate the equations for the boundary conditions
\begin{align*}
A = 
\dfrac{1}{h^2}
\begin{bmatrix}
0
\\
1 & 2  & -1 & & &
\\
  & -1 & 2  & -1 & & 
\\
  &    & -1 & 2 & -1 & & 
\\
 &     &    & \ddots &\ddots & \ddots &
\\
&      &    &        &  -1 & 2 & -1
\\
&      &    &        &  &  & 0
\end{bmatrix}.
\end{align*}